 ![Delta Lake Logo](https://delta.io/_astro/delta-lake-logo.Bqi7mgVq_Kp5oj.webp)
  <br><br> Delta Lake OSS (Open Source Software) is an open-source storage framework designed to bring reliable ACID transactions, scalable metadata handling, and unified batch and streaming data processing to data lakes, enabling the construction of modern "lakehouse" architectures.​

##   Key Features
  **ACID Transactions**: Delta Lake ensures data reliability and consistency by providing serializability, the strongest level of isolation for transactions.​

  **Scalable Metadata**: It efficiently manages petabyte-scale tables and billions of partitions, making large-scale analytics practical.​

  **Time Travel**: Users can access and revert to earlier versions of datasets, supporting auditing and rollbacks.​

  **Schema Enforcement & Evolution**: Delta Lake prevents "bad" data from corrupting datasets and supports gradual schema updates.​

  **Unified Batch/Streaming**: The same table can serve both streaming and batch processing seamlessly.​

  **Openness**: Delta Lake OSS is governed by the Linux Foundation and is community-driven without control by any single company.​

  **Multi-Engine Support**: Works natively with engines like Apache Spark, Flink, Hive, Trino, and Presto, and provides APIs in multiple programming languages (Scala, Java, Python, Rust, Ruby).​

In [0]:
%sql

drop catalog demo_youssefM cascade;
create catalog demo_youssefM;
use catalog demo_youssefM;
create schema delta;
use schema delta;

# Generated Columns & Column Mapping Demo

## What are Generated Columns?

Generated columns in Delta Lake are **computed columns** that are automatically calculated based on other columns in the table using SQL expressions. They provide several key benefits:

* **Data Consistency**: Values are always computed from source data, eliminating inconsistencies
* **Storage Efficiency**: Can be computed on-the-fly or materialized based on needs
* **Data Quality**: Enforce business rules and constraints automatically
* **Query Performance**: Pre-computed values can speed up common queries

## Generated Columns + Column Mapping = Powerful Schema Evolution

When combined with **column mapping**, generated columns become even more powerful:

* ✅ **Safe Schema Changes**: Rename, drop, or modify generated columns without breaking existing data
* ✅ **Flexible Evolution**: Add new generated columns based on existing data
* ✅ **Backward Compatibility**: Existing queries continue to work during schema changes
* ❌ **Without Column Mapping**: Limited ability to modify generated column definitions

Let's explore these capabilities with practical examples!

## 📊 Basic Generated Columns Example

Let's create a sales table with generated columns that automatically compute:
* **Full customer name** from first and last name
* **Total amount** including tax
* **Sales quarter** from the sale date
* **Revenue category** based on amount thresholds

In [0]:
%sql
-- Create a sales table with multiple generated columns
CREATE TABLE IF NOT EXISTS demo_youssefM.delta.sales_with_generated (
  sale_id BIGINT,
  customer_first_name STRING,
  customer_last_name STRING,
  sale_date DATE,
  base_amount DECIMAL(10,2),
  tax_rate DECIMAL(4,3),
  
  -- Generated columns computed from other columns
  full_customer_name STRING GENERATED ALWAYS AS (CONCAT(customer_first_name, ' ', customer_last_name)),
  total_amount DECIMAL(16,5) GENERATED ALWAYS AS (base_amount * (1 + tax_rate)),
  sale_quarter STRING GENERATED ALWAYS AS (CONCAT('Q', QUARTER(sale_date), '-', YEAR(sale_date))),
  revenue_category STRING GENERATED ALWAYS AS (
    CASE 
      WHEN base_amount < 100 THEN 'Small'
      WHEN base_amount < 1000 THEN 'Medium' 
      ELSE 'Large'
    END
  )
) USING DELTA;

In [0]:
%sql
-- Insert sample data - generated columns will be computed automatically
INSERT INTO demo_youssefM.delta.sales_with_generated 
(sale_id, customer_first_name, customer_last_name, sale_date, base_amount, tax_rate)
VALUES 
  (1, 'John', 'Smith', '2024-01-15', 150.00, 0.08),
  (2, 'Sarah', 'Johnson', '2024-03-22', 2500.00, 0.08),
  (3, 'Mike', 'Brown', '2024-06-10', 75.50, 0.08),
  (4, 'Lisa', 'Davis', '2024-09-05', 1200.00, 0.08),
  (5, 'Tom', 'Wilson', '2024-12-18', 45.00, 0.08);

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
select * from demo_youssefM.delta.sales_with_generated 

sale_id,customer_first_name,customer_last_name,sale_date,base_amount,tax_rate,full_customer_name,total_amount,sale_quarter,revenue_category
1,John,Smith,2024-01-15,150.00,0.080,John Smith,162.00000,Q1-2024,Medium
2,Sarah,Johnson,2024-03-22,2500.00,0.080,Sarah Johnson,2700.00000,Q1-2024,Large
3,Mike,Brown,2024-06-10,75.50,0.080,Mike Brown,81.54000,Q2-2024,Small
4,Lisa,Davis,2024-09-05,1200.00,0.080,Lisa Davis,1296.00000,Q3-2024,Large
5,Tom,Wilson,2024-12-18,45.00,0.080,Tom Wilson,48.60000,Q4-2024,Small


## ⚙️ Generated Columns as Data Quality Enforcers

Generated columns automatically enforce business rules and data consistency:

* **Automatic Validation**: Values are always computed correctly
* **Constraint Enforcement**: Cannot insert inconsistent data
* **Business Logic**: Complex rules applied consistently
* **Data Integrity**: Eliminates human error in calculations

In [0]:
%sql
-- This INSERT will FAIL because we're trying to provide values for generated columns
-- Generated columns must be computed, not manually provided
INSERT INTO demo_youssefM.delta.sales_with_generated 
(sale_id, customer_first_name, customer_last_name, sale_date, base_amount, tax_rate, 
 full_customer_name, total_amount)  -- These generated columns should not be provided
VALUES 
  (6, 'Anna', 'Taylor', '2024-11-20', 300.00, 0.08, 'Wrong Name', 999.99);

---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-4639218933760325>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "-- This INSERT will FAIL because we're trying to provide values for generated columns\n-- Generated columns must be computed, not manually provided\nINSERT INTO demo_youssefM.delta.sales_with_generated \n(sale_id, customer_first_name, customer_last_name, sale_date, base_amount, tax_rate, \n full_customer_name, total_amount)  -- These generated columns should not be provided\nVALUES \n  (6, 'Anna', 'Taylor', '2024-11-20', 300.00, 0.08, 'Wrong Name', 999.99);\n")

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2543, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2541 with self.builtin_trap:
   2542     args = (magic_arg_s, cell)
-> 2543     result = fn(*args, **kwargs)
   2545 # The code belo

In [0]:
%sql
-- This INSERT will SUCCEED - only provide base columns, let generated columns compute
INSERT INTO demo_youssefM.delta.sales_with_generated 
(sale_id, customer_first_name, customer_last_name, sale_date, base_amount, tax_rate)
VALUES 
  (6, 'Anna', 'Taylor', '2024-11-20', 300.00, 0.08),
  (7, 'David', 'Miller', '2024-02-14', 50.00, 0.08);

-- Verify the generated columns were computed correctly
SELECT * FROM demo_youssefM.delta.sales_with_generated WHERE sale_id IN (6, 7);

sale_id,customer_first_name,customer_last_name,sale_date,base_amount,tax_rate,full_customer_name,total_amount,sale_quarter,revenue_category
6,Anna,Taylor,2024-11-20,300.00,0.080,Anna Taylor,324.00000,Q4-2024,Medium
7,David,Miller,2024-02-14,50.00,0.080,David Miller,54.00000,Q1-2024,Small


# Part 2: Identity Columns Demo

## What are Identity Columns?

Identity columns in Delta Lake provide **auto-incrementing unique values** that are automatically assigned when new rows are inserted. They're perfect for:

* **Primary Keys**: Automatically generate unique identifiers
* **Sequence Numbers**: Create ordered sequences without gaps
* **Data Lineage**: Track insertion order and data flow
* **Surrogate Keys**: Generate technical keys independent of business data

## Key Features:

* ✅ **Automatic Generation**: Values assigned automatically on INSERT
* ✅ **Uniqueness Guaranteed**: No duplicate values across the table
* ✅ **Configurable**: Set start value and increment step
* ✅ **Column Mapping Compatible**: Works seamlessly with schema evolution

In [0]:
%sql
-- Create a customer table with an identity column as primary key
CREATE TABLE IF NOT EXISTS demo_youssefM.delta.customers_with_identity (
  customer_id BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1000 INCREMENT BY 1),
  first_name STRING NOT NULL,
  last_name STRING NOT NULL,
  email STRING,
  registration_date DATE,
  
  -- Generated columns work alongside identity columns
  full_name STRING GENERATED ALWAYS AS (CONCAT(first_name, ' ', last_name)),
  email_domain STRING GENERATED ALWAYS AS (SUBSTRING_INDEX(email, '@', -1))
) USING DELTA;

In [0]:
%sql
-- Insert data without specifying customer_id - it will be auto-generated
INSERT INTO demo_youssefM.delta.customers_with_identity 
(first_name, last_name, email, registration_date)
VALUES 
  ('Alice', 'Johnson', 'alice.johnson@email.com', '2024-01-15'),
  ('Bob', 'Smith', 'bob.smith@company.org', '2024-02-20'),
  ('Carol', 'Davis', 'carol.davis@university.edu', '2024-03-10'),
  ('David', 'Wilson', 'david.wilson@startup.io', '2024-04-05');

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
-- See how identity column values were automatically assigned
SELECT 
  customer_id,      -- Identity column: auto-generated starting from 1000
  first_name,
  last_name,
  full_name,        -- Generated column: computed from first + last name
  email,
  email_domain,     -- Generated column: extracted from email
  registration_date
FROM demo_youssefM.delta.customers_with_identity
ORDER BY customer_id;

customer_id,first_name,last_name,full_name,email,email_domain,registration_date
1000,Alice,Johnson,Alice Johnson,alice.johnson@email.com,email.com,2024-01-15
1001,Bob,Smith,Bob Smith,bob.smith@company.org,company.org,2024-02-20
1002,Carol,Davis,Carol Davis,carol.davis@university.edu,university.edu,2024-03-10
1003,David,Wilson,David Wilson,david.wilson@startup.io,startup.io,2024-04-05


## 🚀 The Power Combination: Identity + Generated + Column Mapping

Let's create a comprehensive example showing all three features working together:

* **Identity Columns**: Auto-incrementing primary keys
* **Generated Columns**: Computed business logic
* **Column Mapping**: Safe schema evolution

This combination provides the ultimate flexibility for modern data architectures!

In [0]:
%sql
-- Create an orders table with identity, generated columns, and column mapping
CREATE TABLE IF NOT EXISTS demo_youssefM.delta.orders_advanced (
  -- Identity column for unique order IDs
  order_id BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 10000 INCREMENT BY 1),
  
  -- Base columns
  customer_id BIGINT,
  product_name STRING,
  quantity INT,
  unit_price DECIMAL(10,2),
  order_date TIMESTAMP,
  
  -- Generated columns for business logic
  line_total DECIMAL(21,2) GENERATED ALWAYS AS (quantity * unit_price),
  order_year INT GENERATED ALWAYS AS (YEAR(order_date)),
  order_month STRING GENERATED ALWAYS AS (DATE_FORMAT(order_date, 'yyyy-MM')),
  price_category STRING GENERATED ALWAYS AS (
    CASE 
      WHEN unit_price < 10 THEN 'Budget'
      WHEN unit_price < 100 THEN 'Standard'
      ELSE 'Premium'
    END
  ),
  order_size STRING GENERATED ALWAYS AS (
    CASE 
      WHEN quantity <= 1 THEN 'Single'
      WHEN quantity <= 5 THEN 'Small Batch'
      WHEN quantity <= 20 THEN 'Medium Batch'
      ELSE 'Large Batch'
    END
  )
) USING DELTA
TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name',
  'delta.minReaderVersion' = '2',
  'delta.minWriterVersion' = '5'
);

In [0]:
%sql
-- Insert sample orders - order_id will be auto-generated, computed columns calculated
INSERT INTO demo_youssefM.delta.orders_advanced 
(customer_id, product_name, quantity, unit_price, order_date)
VALUES 
  (1001, 'Laptop Pro', 1, 1299.99, '2024-01-15 10:30:00'),
  (1002, 'Wireless Mouse', 3, 29.99, '2024-01-16 14:20:00'),
  (1003, 'Monitor 4K', 2, 399.99, '2024-02-01 09:15:00'),
  (1001, 'USB Cable', 10, 9.99, '2024-02-15 16:45:00'),
  (1004, 'Keyboard Mechanical', 1, 149.99, '2024-03-01 11:00:00'),
  (1002, 'Webcam HD', 25, 79.99, '2024-03-10 13:30:00');

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
-- See all features working together
SELECT 
  order_id,         -- Identity: auto-generated unique ID
  customer_id,
  product_name,
  quantity,
  unit_price,
  line_total,       -- Generated: quantity * unit_price
  order_date,
  order_year,       -- Generated: extracted year
  order_month,      -- Generated: formatted month
  price_category,   -- Generated: price tier
  order_size        -- Generated: quantity category
FROM demo_youssefM.delta.orders_advanced
ORDER BY order_id;

order_id,customer_id,product_name,quantity,unit_price,line_total,order_date,order_year,order_month,price_category,order_size
10000,1001,Laptop Pro,1,1299.99,1299.99,2024-01-15T10:30:00.000Z,2024,2024-01,Premium,Single
10001,1002,Wireless Mouse,3,29.99,89.97,2024-01-16T14:20:00.000Z,2024,2024-01,Standard,Small Batch
10002,1003,Monitor 4K,2,399.99,799.98,2024-02-01T09:15:00.000Z,2024,2024-02,Premium,Small Batch
10003,1001,USB Cable,10,9.99,99.90,2024-02-15T16:45:00.000Z,2024,2024-02,Budget,Medium Batch
10004,1004,Keyboard Mechanical,1,149.99,149.99,2024-03-01T11:00:00.000Z,2024,2024-03,Premium,Single
10005,1002,Webcam HD,25,79.99,1999.75,2024-03-10T13:30:00.000Z,2024,2024-03,Standard,Large Batch


# 🏆 Summary: Generated Columns, Identity Columns & Column Mapping

## What We've Demonstrated

### 📊 **Generated Columns**
* **Automatic Computation**: Values calculated from other columns using SQL expressions
* **Data Quality**: Enforce business rules and prevent inconsistent data
* **Performance**: Pre-computed values for faster queries
* **Examples**: Full names, totals, categories, date extractions

### 🆔 **Identity Columns** 
* **Auto-Incrementing**: Unique sequential values generated automatically
* **Primary Keys**: Perfect for surrogate keys and unique identifiers
* **Configurable**: Set start values and increment steps
* **Guaranteed Uniqueness**: No duplicate values across the table

### 🔄 **Column Mapping**
* **Schema Evolution**: Safely rename, drop, or modify columns
* **Backward Compatibility**: Existing queries continue to work
* **Future-Proof**: Enable flexible schema changes over time

## ✨ **The Power Combination**

When used together, these features provide:

1. **🛡️ Data Integrity**: Identity columns for unique keys + generated columns for consistent calculations
2. **🚀 Performance**: Pre-computed values reduce query complexity
3. **🔧 Flexibility**: Column mapping enables safe schema evolution
4. **💼 Business Logic**: Embedded rules ensure data quality
5. **🔒 Future-Proof**: Adapt to changing requirements without breaking existing systems

## 📝 **Best Practices**

* **Always enable column mapping** for new tables that may evolve
* **Use identity columns** for primary keys instead of manual ID management
* **Leverage generated columns** for business rules and computed values
* **Test schema changes** in development before production
* **Document column purposes** and generation logic for team clarity

---

✅ **Result**: A robust, scalable, and maintainable data architecture that grows with your business needs!